In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/GitHub/Biological-Foundation-Model/Notebooks')
!pip install -r ../requirements.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd "/content/drive/My Drive/Colab Notebooks/huggingface_models/Geneformer"
!pip install .
from geneformer import TOKEN_DICTIONARY_FILE
from geneformer import perturber_utils as pu
from geneformer import TranscriptomeTokenizer
os.chdir('/content/drive/MyDrive/GitHub/Biological-Foundation-Model/Notebooks')

/content/drive/My Drive/Colab Notebooks/huggingface_models/Geneformer
Processing /content/drive/My Drive/Colab Notebooks/huggingface_models/Geneformer
  Preparing metadata (setup.py) ... done
  Created wheel for geneformer: filename=geneformer-0.1.0-py3-none-any.whl size=2410396 sha256=8a74e1beeac1ec7f3d6a68eccec25736a7ff5d9521ecf8a7abd9a1d9f92fba95
  Stored in directory: /tmp/pip-ephem-wheel-cache-cgd2g99w/wheels/11/0c/61/3c7cc3be8408c2e8e59120ac283f66cb87fec65d8e501d2ed3
Successfully built geneformer
  Attempting uninstall: geneformer
    Found existing installation: geneformer 0.1.0
    Uninstalling geneformer-0.1.0:
      Successfully uninstalled geneformer-0.1.0


In [3]:
## prepare data
from datasets import load_dataset, load_from_disk
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np
import pandas as pd

from geneformer import TOKEN_DICTIONARY_FILE
from geneformer import perturber_utils as pu
from geneformer import TranscriptomeTokenizer

# load the training data and the tokenizer from Geneformer
dataset = load_from_disk("/content/drive/My Drive/Colab Notebooks/datasets/geneformer/cell_type_train_data.dataset") # 249556 data points
with open("/content/drive/My Drive/Colab Notebooks/huggingface_models/Geneformer/geneformer/gene_dictionaries_30m/token_dictionary_gc30M.pkl", "rb") as f:
    token_dict = pickle.load(f)


# load model
model = pu.load_model("Pretrained", num_classes = 0, model_directory = "/content/drive/My Drive/Colab Notebooks/huggingface_models/Geneformer/gf-12L-30M-i2048", mode="eval")